In [19]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import utils
from PIL import Image
import csv
from torch.autograd import Variable
checkpoint_path = './checkpoints'
data_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def label_num_counter(ls):
    lbs = {}
    for line in ls:
        for label in line[1:]:
            if label not in lbs.keys():
                lbs[label] = 1
            else:
                lbs[label] += 1
    return lbs

In [20]:
model, _, _ = utils.load_checkpoint(utils.get_latest_checkpoint(checkpoint_path))
model.eval()
in_features = model.fc[0].in_features
model.fc = nn.Linear(in_features, in_features)
torch.nn.init.eye_(model.fc.weight.requires_grad_(False))

loading latest checkpoint ./checkpoints\cv_20.pth


Parameter containing:
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [25]:
import os

import numpy as np
import pickle

features_dir = './features'
images_dir = './imgs'
features_dict = {}
for img_name in os.listdir(images_dir):
    img_path = os.path.join(images_dir, img_name)
    img = Image.open(img_path)
    img = data_transforms(img)
    
    for param in model.parameters():
        param.requires_grad_(False)
    
    x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)
    y = model(x)
    y = y.data.numpy()
    features_dict[img_name] = y
    # np.save(f'{os.path.join(features_dir, img_name)}.npy', y)
with open('./features/features.pkl', 'wb') as fp:
    pickle.dump(features_dict, fp)